In [30]:
import os
import json
import re

In [108]:
file_path = r"E:\AI_HUB\data\Sublabel\SbL\json\e00139302012p0010.json"

with open(file_path, 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

data

{'info': {'DATASET_NM': '2023년 여행로그 관광사진 데이터',
  'DATASET_DETAIL': '2023년 국내 여행로그 데이터수집으로 구축된 관광사진 데이터'},
 'images': {'PHOTO_FILE_ID': 'e00139302012p0010',
  'PHOTO_FILE_NM': 'e00139302012p0010.jpg',
  'PHOTO_FILE_SAVE_PATH': '/photo/',
  'PHOTO_FILE_RESOLUTION': '4032x3024',
  'PHOTO_FILE_DT': '2023-06-03 15:18:22',
  'PHOTO_FILE_X_COORD': '126.59598611111',
  'PHOTO_FILE_Y_COORD': '37.471613888889',
  'VISIT_AREA_NM': '월미문화의 거리',
  'LANDMARK': ''},
 'caption': {'IMG_CAPTION': '아래쪽에 건물이 있고 건물 뒤쪽에 대관람차가 있습니다',
  'TOKEN': 7,
  'TIME_STAMP': '2023-06-03 15:18:22'},
 'licenses': {'ID': 'CC-BY-SA-4.0', 'NAME': '(주)데이터웨이'}}

In [22]:
# # 테스트
# from konlpy.tag import Okt
# 
# nlp = Okt()
# 
# captions = ["가운데에 조형물과 꽃들과 돌담이 있고 위쪽에 나무들이 있습니다"]
# 
# for caption in captions:
#     keywords = nlp.nouns(caption)
#     print(f"Keywords for '{caption}': {keywords}")

Keywords for '가운데에 조형물과 꽃들과 돌담이 있고 위쪽에 나무들이 있습니다': ['가운데', '조형', '물', '꽃', '돌담', '위쪽', '나무']


In [109]:
# 3066개의 JSON 파일 불러오기

json_foler = r"E:\AI_HUB\data\Sublabel\SbL\json"

extracted_data = []

for filename in os.listdir(json_foler):
    filepath = os.path.join(json_foler, filename)
    with open(filepath, "r", encoding='utf-8-sig') as file:
        data = json.load(file)
        
        # 필요한 정보만 추출
        photo_file_name = data['images']["PHOTO_FILE_NM"]
        visit_area_name = data["images"]["VISIT_AREA_NM"]
        resolution = data["images"]["PHOTO_FILE_RESOLUTION"]
        caption = data["caption"]["IMG_CAPTION"]
        time_stamp = data["caption"]["TIME_STAMP"]
        x_coord = data["images"]["PHOTO_FILE_X_COORD"]
        y_coord = data["images"]["PHOTO_FILE_Y_COORD"]

        
        # 추출 데이터 저장
        extracted_data.append({
            "PHOTO_FILE_NM": photo_file_name,
            "VISIT_AREA_NM": visit_area_name,
            "PHOTO_FILE_RESOLUTION": resolution,
            "CAPTION": caption,
            "TIME_STAMP": time_stamp,
            "PHOTO_FILE_X_COORD": x_coord,
            "PHOTO_FILE_Y_COORD": y_coord
            })
# 결과 확인
print(f"총 {len(extracted_data)}개의 JSON 파일에서 데이터를 추출했습니다.")
print(extracted_data[:3])  # 일부 결과 출력

총 3066개의 JSON 파일에서 데이터를 추출했습니다.
[{'PHOTO_FILE_NM': 'e00049901099p0002.jpg', 'VISIT_AREA_NM': '미리 스테이', 'PHOTO_FILE_RESOLUTION': '4032x3024', 'CAPTION': '아래쪽에 잔디밭과 길이 있고 잔디밭 위에 조형물과 나무들이 있습니다', 'TIME_STAMP': '2023-06-05 17:26:06', 'PHOTO_FILE_X_COORD': '126.37036111111', 'PHOTO_FILE_Y_COORD': '37.679913888889'}, {'PHOTO_FILE_NM': 'e00042601005p0010.jpg', 'VISIT_AREA_NM': '', 'PHOTO_FILE_RESOLUTION': '4000x3000', 'CAPTION': '가운데에 잔디밭과 울타리가 있고 잔디밭 양쪽에 한옥 건물이 있습니다', 'TIME_STAMP': '2023-06-10 17:45:20', 'PHOTO_FILE_X_COORD': '127.04675249972', 'PHOTO_FILE_Y_COORD': '37.5082894'}, {'PHOTO_FILE_NM': 'e00049301003p0001.jpg', 'VISIT_AREA_NM': '부평향교', 'PHOTO_FILE_RESOLUTION': '4624x3468', 'CAPTION': '가운데에 한옥 건물이 있고 오른쪽에 나무들이 있습니다', 'TIME_STAMP': '2023-06-17 11:21:16', 'PHOTO_FILE_X_COORD': '126.7203051', 'PHOTO_FILE_Y_COORD': '37.539813799722'}]


In [110]:
output_json_path = r"E:\AI_HUB\data\Sublabel\SbL\extracted_data_all_caption.json"
with open(output_json_path, "w", encoding="utf-8-sig") as outfile:
    json.dump(extracted_data, outfile, ensure_ascii=False, indent=4)
print(f"데이터가 {output_json_path}에 저장되었습니다.")

데이터가 E:\AI_HUB\data\Sublabel\SbL\extracted_data_all_caption.json에 저장되었습니다.


In [102]:
from konlpy.tag import Okt
import re

# 형태소 분석기 로드
okt = Okt()

# 위치 정보를 나타내는 정규식 패턴
location_pattern = re.compile(r"^(가운데|위쪽|아래쪽|오른쪽|왼쪽|좌측|우측|여러|개|앞쪽|양쪽|위|아래|앞|뒤|뒤쪽|중앙|앞쪽|위로|아래로)$")

# 조사 패턴 정의
josa_pattern = re.compile(r"(이|가|에|의|와|과|으로|로|에서|도|지|들이)$")

# 명사 사전 적용 함수
def apply_noun_dict_to_caption(caption, noun_dict):
    words = caption.split()  
    refined_words = [noun_dict.get(word, word) for word in words] 
    return " ".join(refined_words)  

# 명사 사전
known_compounds = {
    "건물이": "건물", 
    "조형물이": "조형물", 
    "물이": "물", 
    "건축물이": "건축물", 
    "길이": "길",
    "돌이": "돌",
    "표지가": "표지판", 
    "꽃들이": "꽃",
    "철문과": "철문", 
    "출입문과": "출입문", 
    "한옥문과": "한옥문",
    "창문과": "창문", 
    "문과": "문", 
    "초가": "초가집", 
    "풀이": "풀",
    "울타리가": "울타리",  
    "다리가": "다리", 
    "등대가": "등대", 
    "분수대가": "분수대", 
    "난간대가": "난간대", 
    "계양대가": "계양대", 
    "전망대가": "전망대", 
    "첨성대가": "첨성대", 
    "전봇대가": "전봇대", 
    "방파제가": "방파제",
    "가판대가": "가판대", 
    "풍력발전소가": "풍력발전소",
    "입간판이": "입간판",  
    "산이": "산",
    "보도": "보도블록",
    "안내": "안내판",
    "조형": "조형물",
    "옥": "한옥",
    
}

# 데이터 처리
for item in extracted_data:
    original_caption = item["CAPTION"]
    
    # 1. 명사 사전 적용
    refined_caption = apply_noun_dict_to_caption(original_caption, known_compounds)
    item["REFINED_CAPTION"] = refined_caption  # 새로운 필드에 저장
    
    # 2. 명사만 추출
    nouns = okt.nouns(refined_caption)
    
    # 3. 조사 제거
    nouns_without_josa = [re.sub(josa_pattern, "", word) for word in nouns]
    
    # 4. 위치 정보 제거
    filtered_keywords = [word for word in nouns_without_josa if not location_pattern.match(word)]
    
    # 5. 필터링된 키워드 저장
    item["FILTERED_KEYWORDS"] = filtered_keywords

# 결과 확인
for item in extracted_data[:5]:  # 첫 5개 데이터 확인
    print(f"Original: {item['CAPTION']}")
    print(f"Refined: {item['REFINED_CAPTION']}")
    print(f"Filtered Keywords: {item['FILTERED_KEYWORDS']}")
    print("-" * 50)

Original: 아래쪽에 잔디밭과 길이 있고 잔디밭 위에 조형물과 나무들이 있습니다
Refined: 아래쪽에 잔디밭과 길 있고 잔디밭 위에 조형물과 나무들이 있습니다
Filtered Keywords: ['잔디밭', '길', '잔디밭', '조형', '물', '나무']
--------------------------------------------------
Original: 가운데에 잔디밭과 울타리가 있고 잔디밭 양쪽에 한옥 건물이 있습니다
Refined: 가운데에 잔디밭과 울타리 있고 잔디밭 양쪽에 한옥 건물 있습니다
Filtered Keywords: ['잔디밭', '울타리', '잔디밭', '옥', '건물']
--------------------------------------------------
Original: 가운데에 한옥 건물이 있고 오른쪽에 나무들이 있습니다
Refined: 가운데에 한옥 건물 있고 오른쪽에 나무들이 있습니다
Filtered Keywords: ['옥', '건물', '나무']
--------------------------------------------------
Original: 아래쪽에 돌바닥이 있고 가운데에 나무와 잔디가 있습니다
Refined: 아래쪽에 돌바닥이 있고 가운데에 나무와 잔디가 있습니다
Filtered Keywords: ['돌', '바닥', '나무', '잔디']
--------------------------------------------------
Original: 가운데에 길이 있고 길 양쪽에 나무들이 있습니다
Refined: 가운데에 길 있고 길 양쪽에 나무들이 있습니다
Filtered Keywords: ['길', '길', '나무']
--------------------------------------------------


In [98]:
# # 특정 단어를 필터링하여 원문 캡션 출력 --> 문맥 파악 위함
# target_keyword = "옥"  # 특정 단어 설정
# print(f"조사 포함 단어: {target_keyword}")
# for item in extracted_data:
#     if target_keyword in item["CAPTION"]:  # 특정 단어가 포함된 캡션 확인
#         print(f"  원문 캡션: {item['CAPTION']}")

조사 포함 단어: 옥
  원문 캡션: 가운데에 잔디밭과 울타리가 있고 잔디밭 양쪽에 한옥 건물이 있습니다
  원문 캡션: 가운데에 한옥 건물이 있고 오른쪽에 나무들이 있습니다
  원문 캡션: 가운데에 한옥 문이 있고 아래쪽 중앙에 계단이 있고 계단 양쪽에 나무들이 있습니다
  원문 캡션: 중앙에 한옥 건물이 있고 아래에는 길이 있고 한옥 건물 뒤로 나무들이 있습니다
  원문 캡션: 중앙에 한옥 건물이 있고 왼쪽에 담과 풀이 있고 건물 앞에 도로와 돌로 된 안내판이 있습니다
  원문 캡션: 중앙에 한옥이 있고 좌측 아래쪽에는 흙으로 된 바닥이 있고 우측 아래쪽에 나무가 있습니다
  원문 캡션: 중앙에 나무들과 한옥이 있고 한옥 아래는 물이 있고 오른쪽 아래에 울타리가 있습니다
  원문 캡션: 왼쪽에 한옥 건물이 있고 오른쪽에 나무가 있고 나무 뒤에 한옥 건물이 있습니다
  원문 캡션: 가운데에 한옥 건물이 있고 한옥 건물 뒤쪽에 나무들이 있습니다
  원문 캡션: 아래쪽에 물이 있고 가운데에 한옥 건물이 있습니다
  원문 캡션: 아래쪽에 계단이 있고 계단 위쪽에 한옥문이 있습니다
  원문 캡션: 가운데에 한옥 건물이 있고 한옥 건물 뒤쪽에 언덕과 나무들이 있습니다
  원문 캡션: 좌측에 파라솔과 테이블과 의자들이 있고 우측에 한옥 건물이 있습니다
  원문 캡션: 가운데에 한옥 건물이 있고 아래에는 물이 있고 왼쪽에 나무가 있습니다
  원문 캡션: 오른쪽에 한옥 건물이 있고 왼쪽에 나무들이 있고 아래쪽에 물이 있습니다
  원문 캡션: 왼쪽에 한옥 건물이 있고 오른쪽에 나무들이 있고 아래쪽에 물이 있습니다
  원문 캡션: 아래쪽에 물이 있고 좌측에 나무들이 있고 위쪽에 한옥 건물들이 있습니다
  원문 캡션: 우측에 나무들이 있고 가운데에 한옥 건물이 있고 건물 뒤쪽에 나무들이 있습니다
  원문 캡션: 가운데에 한옥 건물이 있고 한옥 건물 뒤쪽에 나무들이 있습니다
  원문 캡션: 가운데에 한옥 건물이 있고 한옥 건물 앞쪽에 계단이 있고 위쪽에 나무들이 있습니다
  원문

In [104]:
# 중복된 단어 제거
for item in extracted_data:
    if "FILTERED_KEYWORDS" in item:
        item["FILTERED_KEYWORDS"] = list(set(item["FILTERED_KEYWORDS"]))

In [105]:
# 결과 출력
for item in extracted_data:
    print(item["FILTERED_KEYWORDS"])

['물', '조형', '길', '나무', '잔디밭']
['잔디밭', '울타리', '건물', '옥']
['옥', '건물', '나무']
['잔디', '나무', '돌', '바닥']
['나무', '길']
['물', '조형', '건물', '돌', '나무', '잔디밭', '울타리']
['꽃', '풀', '나무', '길']
['돌', '나무', '기와집', '바닥']
['옥', '문', '계단', '나무']
['옥', '건물', '나무', '길']
['물레방아', '산', '길', '인공', '나무', '울타리', '꽃', '폭포']
['건물', '산', '나무', '파라솔', '꽃']
['건물', '자갈밭', '의자', '풀', '나무']
['갯벌', '바위', '바다']
['옥', '건물', '도', '돌', '풀', '판', '담', '안내']
['담장', '건물', '풀', '나무']
['건물', '나무']
['옥', '바닥', '나무', '흙']
['물', '옥', '울타리', '나무']
['울타리', '산', '나무']
['옥', '건물', '나무']
['건물', '의자']
['물', '방파제', '등대']
['정자', '꽃', '산', '길']
['건축물', '나무', '길']
['건물', '계단']
['물', '조형', '나무']
['연등', '건물']
['나무', '길']
['물', '조형', '간판', '풀']
['연등', '계단']
['돌담', '나무', '길']
['물', '조형', '길']
['불상', '나무']
['물', '나무', '길']
['풀', '나무']
['나무', '길']
['건물', '나무']
['동물', '풀', '자갈']
['물', '건물', '나무']
['물', '조형', '나무']
['건물', '동물']
['끝', '건물', '나무', '길']
['담장', '건물', '도']
['옥', '건물', '나무']
['물', '옥', '건물']
['물', '조형', '건물']
['물', '조형', '건물', '나무']
['연등', '조

In [106]:
# 쓸모 없는 단어 수동 제거
stopwords = ["도", "끝", "끝쪽", "배경", "위쪽"]
# FILTERED_KEYWORDS에서 불용어 제거
for item in extracted_data:
    if "FILTERED_KEYWORDS" in item:
        item["FILTERED_KEYWORDS"] = [
            word for word in item["FILTERED_KEYWORDS"] if word not in stopwords
        ]

In [107]:
output_json_path = r"E:\AI_HUB\data\Sublabel\SbL\extracted_data.json"
with open(output_json_path, "w", encoding="utf-8-sig") as outfile:
    json.dump(extracted_data, outfile, ensure_ascii=False, indent=4)
print(f"데이터가 {output_json_path}에 저장되었습니다.")

데이터가 E:\AI_HUB\data\Sublabel\SbL\extracted_data.json에 저장되었습니다.
